In [1]:
import psycopg2
import names

In [2]:
%run functions.ipynb

In [3]:
database_setup = {
    'host' : '127.0.0.11'
    ,'database': 'ZSBD_p1_restore'
    ,'user' : 'inzdanych123'
    ,'password' : 'inzdanych123'
    ,'port' : '5432'
}

In [4]:
def do_query(query, print_out = False, commit = False):
    try:
        con = psycopg2.connect(
                    host = database_setup['host'],
                    database = database_setup['database'],
                    user = database_setup['user'],
                    password = database_setup['password'],
                    port = database_setup['port'])

        cur = con.cursor()

        cur.execute(query)
        
        for notice in con.notices:
            print(notice)

        if commit:
            con.commit()

        if print_out:
            column_names = [desc[0] for desc in cur.description]
            row_list = []
            row = cur.fetchone()
            while row is not None:
                row_list.append(row)
                #print(row)
                row = cur.fetchone()
            df = pd.DataFrame(row_list, columns=column_names)
            print(df)
        
        cur.close()
        con.close()
        print('\n')
    except (Exception, psycopg2.DatabaseError) as error:
            print(error)

    finally:
        if con is not None:
            con.close()

In [5]:
do_query("""
CREATE OR REPLACE VIEW view_holder.leki AS
SELECT coalesce("Aktualnie przyjmowane leki".data_zakonczenia_ventolin__asthma, cast(now() as date)) -"Aktualnie przyjmowane leki".data_rozpoczecia_ventolin__asthma as "dni_brania_ventolin",
pacjent.imie, pacjent.nazwisko, extract(year from age(now(), pacjent.data_urodzenia)) as wiek_pacjenta, 
"Aktualnie przyjmowane leki".data_rozpoczecia_ventolin__asthma, "Aktualnie przyjmowane leki".data_zakonczenia_ventolin__asthma
FROM ai_med.pacjent INNER JOIN ai_med."Aktualnie przyjmowane leki" 
ON "Aktualnie przyjmowane leki".id = pacjent.id 
WHERE DATE_TRUNC('day',"Aktualnie przyjmowane leki".data_rozpoczecia_ventolin__asthma) >= CURRENT_DATE - interval '4 year' 
ORDER BY dni_brania_ventolin;
"""
,commit=True
,print_out=True
)

'NoneType' object is not iterable


In [6]:
do_query("""CREATE OR REPLACE VIEW view_holder.wyniki_morfologii_dla_brak_objawow AS
SELECT pacjent.imie, pacjent.nazwisko, extract(year from age(now(), pacjent.data_urodzenia)) as wiek_pacjenta
,morfologia.ilosc_erytrocytow, morfologia.stezenie_hemoglobiny , morfologia.mcv, morfologia.mchc, morfologia.mch, morfologia.rdw, morfologia.data_wykonanej_morfologii
--,objawy.bol_glowy, objawy.zmeczenie
FROM ai_med.pacjent 
INNER JOIN ai_med.objawy ON objawy.id = pacjent.id
INNER JOIN ai_med.morfologia ON morfologia.pacjentid = pacjent.id
WHERE objawy.bol_glowy = FALSE AND objawy.zmeczenie = FALSE
ORDER BY objawy.bol_glowy;"""
,commit=True
,print_out=True
)

'NoneType' object is not iterable


In [7]:
query_list = ['SELECT * FROM view_holder.wyniki_morfologii_dla_brak_objawow;',
'SELECT * FROM view_holder.leki;', 
'SELECT * from ai_med.pacjent',
"INSERT INTO ai_med.Pacjent(imie, nazwisko, data_urodzenia) VALUES ('%s', '%s', '%s');" %(names.get_first_name(), names.get_last_name(), random_date(datetime.date(1920, 1, 1), datetime.date(2004, 2, 1))),
'CREATE SCHEMA IF NOT EXISTS AI_med;']

In [8]:
def user_test(query_list, user = "inzdanych123", password = "inzdanych123"):
    print(user + " test")
    for query in query_list:
        try:
            con = psycopg2.connect(
                            host = database_setup['host'],
                            database = database_setup['database'],
                            user = user,
                            password = password,
                            port = database_setup['port'])

            cur = con.cursor()
        
            cur.execute(query)
            print("Jest zgoda na: " + query, "\n") 
            for notice in con.notices:
                print(notice)
        except psycopg2.Error as e:
            print("Nie ma zgody na: " + query)
            print(e, "\n")

    


    cur.close()
    con.close()

In [9]:
user_test(query_list, "inzdanych123", "inzdanych123")

inzdanych123 test
Jest zgoda na: SELECT * FROM view_holder.wyniki_morfologii_dla_brak_objawow; 

Jest zgoda na: SELECT * FROM view_holder.leki; 

Jest zgoda na: SELECT * from ai_med.pacjent 

Jest zgoda na: INSERT INTO ai_med.Pacjent(imie, nazwisko, data_urodzenia) VALUES ('Rob', 'Gonzales', '1952-07-03'); 

Jest zgoda na: CREATE SCHEMA IF NOT EXISTS AI_med; 

NOTICE:  schema "ai_med" already exists, skipping



In [10]:
user_test(query_list, "inzml123", "inzml123")

inzml123 test
Nie ma zgody na: SELECT * FROM view_holder.wyniki_morfologii_dla_brak_objawow;
permission denied for schema view_holder
LINE 1: SELECT * FROM view_holder.wyniki_morfologii_dla_brak_objawow...
                      ^
 

Nie ma zgody na: SELECT * FROM view_holder.leki;
permission denied for schema view_holder
LINE 1: SELECT * FROM view_holder.leki;
                      ^
 

Nie ma zgody na: SELECT * from ai_med.pacjent
permission denied for schema ai_med
LINE 1: SELECT * from ai_med.pacjent
                      ^
 

Nie ma zgody na: INSERT INTO ai_med.Pacjent(imie, nazwisko, data_urodzenia) VALUES ('Rob', 'Gonzales', '1952-07-03');
permission denied for schema ai_med
LINE 1: INSERT INTO ai_med.Pacjent(imie, nazwisko, data_urodzenia) V...
                    ^
 

Nie ma zgody na: CREATE SCHEMA IF NOT EXISTS AI_med;
permission denied for database ZSBD_p1_restore
 



In [11]:
user_test(query_list, "lekarz123", "lekarz123")

lekarz123 test
Nie ma zgody na: SELECT * FROM view_holder.wyniki_morfologii_dla_brak_objawow;
permission denied for schema view_holder
LINE 1: SELECT * FROM view_holder.wyniki_morfologii_dla_brak_objawow...
                      ^
 

Nie ma zgody na: SELECT * FROM view_holder.leki;
permission denied for schema view_holder
LINE 1: SELECT * FROM view_holder.leki;
                      ^
 

Nie ma zgody na: SELECT * from ai_med.pacjent
permission denied for schema ai_med
LINE 1: SELECT * from ai_med.pacjent
                      ^
 

Nie ma zgody na: INSERT INTO ai_med.Pacjent(imie, nazwisko, data_urodzenia) VALUES ('Rob', 'Gonzales', '1952-07-03');
permission denied for schema ai_med
LINE 1: INSERT INTO ai_med.Pacjent(imie, nazwisko, data_urodzenia) V...
                    ^
 

Nie ma zgody na: CREATE SCHEMA IF NOT EXISTS AI_med;
permission denied for database ZSBD_p1_restore
 



In [12]:
user_test(query_list, "pacjent123", "pacjent123")

pacjent123 test
Nie ma zgody na: SELECT * FROM view_holder.wyniki_morfologii_dla_brak_objawow;
permission denied for schema view_holder
LINE 1: SELECT * FROM view_holder.wyniki_morfologii_dla_brak_objawow...
                      ^
 

Nie ma zgody na: SELECT * FROM view_holder.leki;
permission denied for schema view_holder
LINE 1: SELECT * FROM view_holder.leki;
                      ^
 

Nie ma zgody na: SELECT * from ai_med.pacjent
permission denied for schema ai_med
LINE 1: SELECT * from ai_med.pacjent
                      ^
 

Nie ma zgody na: INSERT INTO ai_med.Pacjent(imie, nazwisko, data_urodzenia) VALUES ('Rob', 'Gonzales', '1952-07-03');
permission denied for schema ai_med
LINE 1: INSERT INTO ai_med.Pacjent(imie, nazwisko, data_urodzenia) V...
                    ^
 

Nie ma zgody na: CREATE SCHEMA IF NOT EXISTS AI_med;
permission denied for database ZSBD_p1_restore
 



In [13]:
user_test(query_list, "lekarz_perspektywa", "lekarz_perspektywa")

lekarz_perspektywa test
Jest zgoda na: SELECT * FROM view_holder.wyniki_morfologii_dla_brak_objawow; 

Jest zgoda na: SELECT * FROM view_holder.leki; 

Nie ma zgody na: SELECT * from ai_med.pacjent
permission denied for schema ai_med
LINE 1: SELECT * from ai_med.pacjent
                      ^
 

Nie ma zgody na: INSERT INTO ai_med.Pacjent(imie, nazwisko, data_urodzenia) VALUES ('Rob', 'Gonzales', '1952-07-03');
permission denied for schema ai_med
LINE 1: INSERT INTO ai_med.Pacjent(imie, nazwisko, data_urodzenia) V...
                    ^
 

Nie ma zgody na: CREATE SCHEMA IF NOT EXISTS AI_med;
permission denied for database ZSBD_p1_restore
 

